<a href="https://colab.research.google.com/github/nmningmei/Deep_learning_fMRI_EEG/blob/master/10_1_searchlight_representational_similarity_analysis%2C_bonus_decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The script to illustrate a way to perform searchlight RSA.

# Get the extracted features and the mask files

In [1]:
try:
    !git clone https://github.com/nmningmei/Extracted_features_of_Spanish_image_dataset.git
except:
    !ls Extracted_features_of_Spanish_image_dataset

fatal: destination path 'Extracted_features_of_Spanish_image_dataset' already exists and is not an empty directory.


# Get the fMRI data

In [2]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1vLWSdXpOxqp3jOCypsWA27AEptsGdwav'
downloaded = drive.CreateFile({'id': file_id})


In [3]:
downloaded.GetContentFile('whole_bran.tar.gz')

In [4]:
!tar -xvf whole_bran.tar.gz

whole_brain_conscious.csv
whole_brain_unconscious.csv
whole_brain_unconscious.nii.gz
whole_brain_conscious.nii.gz


In [5]:
ls

adc.json                                      whole_brain_conscious.nii.gz*
Extracted_features_of_Spanish_image_dataset/  whole_brain_unconscious.csv*
RSA_unconscious.nii.gz                        whole_brain_unconscious.nii.gz*
sample_data/                                  whole_bran.tar.gz
whole_brain_conscious.csv*


# install and update some of the libraries if not 

In [6]:
try:
    from nilearn.input_data import NiftiMasker
    from nilearn.image import new_img_like
    from brainiak.searchlight.searchlight import Searchlight
    from brainiak.searchlight.searchlight import Ball
except:
    !pip install nilearn
    !python3 -m pip install -U brainiak
    from nilearn.input_data import NiftiMasker
    from nilearn.image import new_img_like
    from brainiak.searchlight.searchlight import Searchlight
    from brainiak.searchlight.searchlight import Ball

import os,gc

import pandas as pd
import numpy as np

from nibabel import load as load_fmri
from joblib  import Parallel,delayed
from scipy.spatial import distance
from scipy.stats import spearmanr

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
def normalize(data,axis = 1):
    return data - data.mean(axis).reshape(-1,1)
# Define voxel function
def sfn(l, msk, myrad, bcast_var):
    """
    l: BOLD
    msk: mask array
    myrad: not use
    bcast_var: label -- CNN features
    """
    BOLD = l[0][msk,:].T.copy() # vectorize the voxel values in the sphere
    model = bcast_var.copy() # vectorize the RDM
    # pearson correlation
    RDM_X   = distance.pdist(normalize(BOLD),'correlation')
    RDM_y   = distance.pdist(normalize(model),'correlation')
    D,p     = spearmanr(RDM_X,RDM_y)
    return D
def process_csv(file_name = 'whole_brain_conscious.csv'):
    """
    to add some info to the event files to create better cross-validation folds
    """
    df_data = pd.read_csv(file_name)
    df_data['id'] = df_data['session'] * 1000 + df_data['run'] * 100 + df_data['trials']
    df_data = df_data[df_data.columns[1:]]
    return df_data

In [8]:
radius              = 3 # in mm, the data has voxel size of 2.4mm x 2.4mm x 2.4mm
feature_dir         = 'Extracted_features_of_Spanish_image_dataset/computer_vision_features_no_background'
model_name          = 'VGG19'
label_map           = {'Nonliving_Things':[0,1],'Living_Things':[1,0]}
whole_brain_mask    = 'Extracted_features_of_Spanish_image_dataset/combine_BOLD.nii.gz'
average             = True
n_splits            = 10 # recommend to perform the resampling for more than 500 times
n_jobs              = -1

# implementation of resampling

1. when the study is event-related but the events are not balanced, distribution is hard to measure, so resampling could avoid handling the assumption of normal distribution
2. by the law of large number, when the time of resampling is large enough (exhaust the permutation, i.e. 1-2-3, 2-3-1, 3-1-2, 1-3-2, 2-1-3, 3-2-1, or just a large number, like n = 1000), the average of the resampling estimate is an unbias estimate of the population measure, and the 95% credible interval (confidence interval for frequentist statitistics) contains the population with chance of 95%.
3. so, we gather one trial of each unique item, in total 96 trial, for both the BOLD signals (96, 88, 88, 66) and the CNN features (96, 300), and we compute the RDM of the BOLD signals in the searchlight sphere and the RDM of the CNN features regardless of the searchlight sphere. We then correlate the RDM of the BOLD in the searchlight sphere to the RDM of the CNN features.
4. repeat step 3. for different sampling

In [ ]:
for conscious_state in ['unconscious','conscious']:
    np.random.seed(12345)

    df_data         = process_csv(f'whole_brain_{conscious_state}.csv')
    # load the data in the format of numpy but keep the 4D dimensions
    BOLD_image      = load_fmri(f'whole_brain_{conscious_state}.nii.gz')
    print(f'{conscious_state}\tfMRI in {BOLD_image.shape} events in {df_data.shape}')
    targets         = np.array([label_map[item] for item in df_data['targets']])[:,-1]
    # get the image names in the order of the experimental trials
    images          = df_data['paths'].apply(lambda x: x.split('.')[0] + '.npy').values
    # get the CNN features (n_trial x 300)
    CNN_feature     = np.array([np.load(os.path.join(feature_dir,
                                                     model_name,
                                                     item)) for item in images])
    groups          = df_data['labels'].values

    # define a function to create the folds first
    def _proc(df_data):
        """
        This is useful when the number of folds are thousands
        """
        df_picked = df_data.groupby('labels').apply(lambda x: x.sample(n = 1).drop('labels',axis = 1)).reset_index()
        df_picked = df_picked.sort_values(['targets','subcategory','labels'])
        idx_test  = df_picked['level_1'].values
        return idx_test
    print(f'partitioning data for {n_splits} folds')
    idxs = Parallel(n_jobs = -1, verbose = 1)(delayed(_proc)(**{
                'df_data':df_data,}) for _ in range(n_splits))
    gc.collect() # free memory that is occupied by garbage

    # define a function to run the RSA
    def _searchligh_RSA(idx,
                        sl_rad = radius, 
                        max_blk_edge = radius - 1, 
                        shape = Ball,
                        min_active_voxels_proportion = 0,
                        ):
        # Brainiak function
        sl = Searchlight(sl_rad = sl_rad, 
                        max_blk_edge = max_blk_edge, 
                        shape = shape,
                        min_active_voxels_proportion = min_active_voxels_proportion,
                        )
        # distribute the data based on the sphere
        ## the first input is usually the BOLD signal, and it is in the form of 
        ## lists not arrays, representing each subject
        ## the second input is usually the mask, and it is in the form of array
        sl.distribute([np.asanyarray(BOLD_image.dataobj)[:,:,:,idx]], 
                        np.asanyarray(load_fmri(whole_brain_mask).dataobj) == 1)
        # broadcasted data is the data that remains the same during RSA
        sl.broadcast(CNN_feature[idx])
        # run searchlight algorithm
        global_outputs = sl.run_searchlight(sfn,
                                            pool_size = 1, # we run each RSA using a single CPU
                                            )
        return global_outputs
    for _ in range(10):
        gc.collect()
    res = Parallel(n_jobs = -1,verbose = 1,)(delayed(_searchligh_RSA)(**{
                'idx':idx}) for idx in idxs)
    # save the data
    results_to_save = np.zeros(np.concatenate([BOLD_image.shape[:3],[n_splits]]))
    for ii,item in enumerate(res):
        results_to_save[:,:,:,ii] = np.array(item, dtype=np.float)
    results_to_save = new_img_like(BOLD_image,results_to_save,)
    results_to_save.to_filename(f'RSA_{conscious_state}.nii.gz')

unconscious	fMRI in (88, 88, 66, 782) events in (782, 20)
partitioning data for 10 folds


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 18.5min finished


conscious	fMRI in (88, 88, 66, 514) events in (514, 20)
partitioning data for 10 folds


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


# let's modify the code above to make it a searchlight decoding, without using nilearn

In [ ]:
# modify the voxel function for decoding
# like defining the decoder
# like putting the train-test pipeline in place
def sfn(l, msk, myrad, bcast_var):
    """
    l: BOLD
    msk: mask array
    myrad: not use
    bcast_var: label 
    """
    from sklearn.svm import LinearSVC
    from sklearn.calibration import CalibratedClassifierCV
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import roc_auc_score
    BOLD = l[0][msk,:].T.copy() # vectorize the voxel values in the sphere
    targets,idx_train,idx_test = bcast_var
    
    # scaler the data to between 0 and 1, improve SVM decoding
    scaler = MinMaxScaler()
    # default with L2 regularization
    svm = LinearSVC(class_weight = 'balanced',random_state = 12345)
    # make the SVM to produce probabilistic predictions 
    svm = CalibratedClassifierCV(svm, cv = 5)
    pipeline = make_pipeline(scaler,svm)
    pipeline.fit(BOLD[idx_train],targets[idx_train])
    y_pred = pipeline.predict_proba(BOLD[idx_test])[:,-1]
    score = roc_auc_score(targets[idx_test],y_pred)
    return score

In [ ]:
for conscious_state in ['unconscious','conscious']:
    np.random.seed(12345)

    df_data         = process_csv(f'whole_brain_{conscious_state}.csv')
    # load the data in the format of numpy but keep the 4D dimensions
    BOLD_image      = load_fmri(f'whole_brain_{conscious_state}.nii.gz')
    print(f'{conscious_state}\tfMRI in {BOLD_image.shape} events in {df_data.shape}')
    targets         = np.array([label_map[item] for item in df_data['targets']])[:,-1]
    # get the image names in the order of the experimental trials
    images          = df_data['paths'].apply(lambda x: x.split('.')[0] + '.npy').values
    # get the CNN features (n_trial x 300)
    CNN_feature     = np.array([np.load(os.path.join(feature_dir,
                                                     model_name,
                                                     item)) for item in images])
    groups          = df_data['labels'].values
    ############################################################################
    ######## replace this part #################################################
    # define a function to create the folds first
    def _proc(df_data):
        """
        This is useful when the number of folds are thousands
        """
        df_picked = df_data.groupby('labels').apply(lambda x: x.sample(n = 1).drop('labels',axis = 1)).reset_index()
        df_picked = df_picked.sort_values(['targets','subcategory','labels'])
        idx_test  = df_picked['level_1'].values
        return idx_test
    print(f'partitioning data for {n_splits} folds')
    idxs = Parallel(n_jobs = -1, verbose = 1)(delayed(_proc)(**{
                'df_data':df_data,}) for _ in range(n_splits))
    gc.collect() # free memory that is occupied by garbage
    ############################################################################
    ######## with this #########################################################
    from sklearn.model_selection import StratifiedShuffleSplit
    cv = StraitifiedShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 12345)
    idxs_train,idxs_test = [],[]
    for idx_train,idx_test in cv.split(df_data,targets):
        idxs_train.append(idx_train)
        idxs_test.append(idx_test)
    ############################################################################
    ##### end of modification No.1##############################################
    ############################################################################
    # define a function to run the RSA
    def _searchligh_RSA(idx,
                        sl_rad = radius, 
                        max_blk_edge = radius - 1, 
                        shape = Ball,
                        min_active_voxels_proportion = 0,
                        ):
        # get the train,test split of a given fold
        idx_train,idx_test = idx
        # Brainiak function
        sl = Searchlight(sl_rad = sl_rad, 
                        max_blk_edge = max_blk_edge, 
                        shape = shape,
                        min_active_voxels_proportion = min_active_voxels_proportion,
                        )
        # distribute the data based on the sphere
        ## the first input is usually the BOLD signal, and it is in the form of 
        ## lists not arrays, representing each subject
        ## the second input is usually the mask, and it is in the form of array
        sl.distribute([np.asanyarray(BOLD_image.dataobj)], 
                        np.asanyarray(load_fmri(whole_brain_mask).dataobj) == 1)
        ########################################################################
        ##### second modification ##############################################
        # broadcasted data is the data that remains the same during RSA
        sl.broadcast([targets,idx_train,idx_test]) # <-- add the indices of training and testing
        ###### end of modification No.2 ########################################
        # run searchlight algorithm
        global_outputs = sl.run_searchlight(sfn,
                                            pool_size = 1, # we run each RSA using a single CPU
                                            )
        return global_outputs
    for _ in range(10):
        gc.collect()
    res = Parallel(n_jobs = -1,verbose = 1,)(delayed(_searchligh_RSA)(**{
                'idx':idx}) for idx in idxs)
    # save the data
    results_to_save = np.zeros(np.concatenate([BOLD_image.shape[:3],[n_splits]]))
    for ii,item in enumerate(res):
        results_to_save[:,:,:,ii] = np.array(item, dtype=np.float)
    results_to_save = new_img_like(BOLD_image,results_to_save,)
    results_to_save.to_filename(f'searchlight_decoding_{conscious_state}.nii.gz')